### Run the following chunk of code to import any libraries or packages necessary to run the rest of the code.

In [ ]:
import numpy as np
import os as os
import re as re
import pandas as pd

### Run the following chunk of code to calculate the sum of the time averaged (TA) forces (Electrostatic & Hydrophobic), as well as an errorbar for this value using a block bootstrapping procedure.

The following code will calculat the sum of the electrostatic and hydrophobic forces as well as errorbars (Standard deviation) of each quantity using a block boostrapping procedure with 1000 total samples. Note, the user will need to adjust the directory variables to match their directory structure, or adjust their directory structure to match that of the code.

In [ ]:
base_Directory = ""

phos = 0
chain_Length = 150

input_Working_Directory_One = base_Directory + "Inputs/Electrostatic/" + str(phos) + "x_Phos/"
input_Working_Directory_Two = base_Directory + "Inputs/Hydrophobic/" + str(phos) + "x_Phos/"
output_Working_Directory_One = base_Directory + "Outputs/Electrostatic/Mean/" + str(phos) + "x_Phos/"
output_Working_Directory_Two = base_Directory + "Outputs/Hydrophobic/Mean/" + str(phos) + "x_Phos/"
output_Working_Directory_Three = base_Directory + "Outputs/Electrostatic/SD/" + str(phos) + "x_Phos/"
output_Working_Directory_Four = base_Directory + "Outputs/Hydrophobic/SD/" + str(phos) + "x_Phos/"

force_Conversion = 1.661*10**(-12) #N Per kJ/(nm*mol)

concentrations = [10,11,12,13,14,15,16,17,18,19,20]
protein_Counts = [25,28,30,32,36,40,42,44,48,50,54]

for file in os.listdir(input_Working_Directory_One):
    print(file)
    electrostatic_AA_Forces = np.load(input_Working_Directory_One + file)
    hydrophobic_AA_Forces = np.load(input_Working_Directory_Two + file.replace("Electrostatic_Forces", "Hydrophobic_Forces"))

    concentration = int(re.split("_uM",file)[0])
    protein_Count = [protein_Counts[i] for i in range(len(concentrations)) if concentrations[i] == concentration][0]
    AA_Count = protein_Count*chain_Length
    
    hydrophobic_Sum_Total = 0
    electrostatic_Sum_Total = 0
    hydrophobic_Sum_Error = 0
    electrostatic_Sum_Error = 0
    
    rows = np.shape(electrostatic_AA_Forces)[0]
    columns = np.shape(electrostatic_AA_Forces)[1]
    hydrophobic_Terms = []
    electrostatic_Terms = []
    for i in range(rows):
        if i%100 == 0:
            print(i)
        index_One_Protein = int(np.floor(i/chain_Length))
        for j in range(columns):
            index_Two_Protein = int(np.floor(j/chain_Length))
            if index_One_Protein != index_Two_Protein:
                if hydrophobic_AA_Forces[i,j] < 0:
                    hydrophobic_Terms.append(hydrophobic_AA_Forces[i,j])
                electrostatic_Terms.append(electrostatic_AA_Forces[i,j])
                
    series = electrostatic_Terms
    replicates = 10
    blocks = 50
    batches = 100
    batch_Sum_Array = np.zeros((batches,replicates))

    terms_Per_Block = int(len(series)/blocks)

    for batch in np.arange(batches):
        print("Batch " + str(batch))
        replicate_Arrays = np.zeros((replicates,terms_Per_Block*blocks))
        for replicate in np.arange(replicates):
            random_Blocks = np.random.randint(0,blocks,blocks)
            for block in np.arange(len(random_Blocks)):
                replicate_Arrays[replicate,terms_Per_Block*block:terms_Per_Block*(block + 1)] = series[terms_Per_Block*random_Blocks[block]:terms_Per_Block*(random_Blocks[block] + 1)]
        batch_Sum_Array[batch,:] = np.sum(replicate_Arrays, axis = 1)   
    
    
    batch_Mean = np.mean(batch_Sum_Array)
    electrostatic_Sum_Total = batch_Mean*force_Conversion/2
    electrostatic_Sum_Error = (1/2)*np.sqrt(((1)/(replicates*batches))*np.sum((batch_Sum_Array - batch_Mean)**2))*force_Conversion 
        
        
    series = hydrophobic_Terms
    replicates = 10
    blocks = 50
    batches = 100
    batch_Sum_Array = np.zeros((batches,replicates))

    terms_Per_Block = int(len(series)/blocks)

    for batch in np.arange(batches):
        print("Batch " + str(batch))
        replicate_Arrays = np.zeros((replicates,terms_Per_Block*blocks))
        for replicate in np.arange(replicates):
            random_Blocks = np.random.randint(0,blocks,blocks)
            for block in np.arange(len(random_Blocks)):
                replicate_Arrays[replicate,terms_Per_Block*block:terms_Per_Block*(block + 1)] = series[terms_Per_Block*random_Blocks[block]:terms_Per_Block*(random_Blocks[block] + 1)]
        batch_Sum_Array[batch,:] = np.sum(replicate_Arrays, axis = 1) 
        
    batch_Mean = np.mean(batch_Sum_Array)
    hydrophobic_Sum_Total = batch_Mean*force_Conversion/2
    hydrophobic_Sum_Error = (1/2)*np.sqrt(((1)/(replicates*batches))*np.sum((batch_Sum_Array - batch_Mean)**2))*force_Conversion
    
    np.save(output_Working_Directory_One + file.replace("Electrostatic_Forces","Force_Sums_Electrostatic"), electrostatic_Sum_Total)
    np.save(output_Working_Directory_Two + file.replace("Electrostatic_Forces","Force_Sums_Hydrophobic"), hydrophobic_Sum_Total)
    np.save(output_Working_Directory_Three + file.replace("Electrostatic_Forces","Force_Errors_Electrostatic"), electrostatic_Sum_Error)
    np.save(output_Working_Directory_Four + file.replace("Electrostatic_Forces","Force_Errors_Hydrophobic"),hydrophobic_Sum_Error)